In [1]:
from nuq import NuqClassifier

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
def generate_points_on_circle(circle_r=10, circle_x=0, circle_y=0, n_points=10, uniform=True):
    if not uniform:
        alpha = 2 * np.pi * np.random.rand(n_points)
    else:
        alpha = 2 * np.pi * np.linspace(start=0, stop=1, num=n_points)

    x = circle_r * np.cos(alpha) + circle_x
    y = circle_r * np.sin(alpha) + circle_y

    return np.vstack((x, y)).T

In [3]:
X = generate_points_on_circle(n_points=100)

In [4]:
import matplotlib.pyplot as plt
%matplotlib widget

In [5]:
plt.close()
plt.figure()

plt.scatter(X[:, 0], X[:, 1])
plt.scatter(0, 0, color='red')
plt.axis('equal')
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
nuq = NuqClassifier(
    method='all_data',
    tune_bandwidth=False,
    bandwidth=np.array([1., 1.])
)

In [7]:
y = np.zeros(X.shape[0], dtype='int')

In [8]:
nuq.fit(X, y)

NuqClassifier(bandwidth=array([1., 1.]), method='all_data', n_neighbors=100,
              tune_bandwidth=False)

In [9]:
nuq.predict_proba(np.zeros((1, 2)))

{'probs': array([[0.]]), 'probsm1': array([[-inf]])}

In [10]:
nuq.predict_uncertainty(np.zeros((1, 2)), )

{'epistemic': array(18.90485024),
 'aleatoric': -11.512925464970229,
 'total': 18.904850238547855}

In [11]:
np.log(nuq.coeff)

-11.512925464970229

In [17]:
nuq.get_kde(np.zeros((1, 2)))

array([[-50.91893853]])

In [13]:
C = np.log(2.) - 0.75 * np.log(np.pi) - 0.5 * np.log(X.shape[0]) - 0.5 * np.sum(nuq.bandwidth)

In [14]:
C

-3.4679853268211507

In [15]:
0.5 * np.log(nuq.coeff) - 0.5 * nuq.get_kde(np.zeros((1, 2))) + C

array([[16.23502121]])